# Text Recognition with CTC

In [97]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import re
from os.path import exists

In [102]:
import ctc_utils as utils
from importlib import reload 
import warnings

# Ensure we have always the latest state and 
# not the last import in memory
reload(utils)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jpec/opt/anaconda3/envs/tf2/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/45/1rh915g935nggkhvq3fhbn180000gn/T/ipykernel_41519/876068771.py", line 7, in <cell line: 7>
    reload(utils)
  File "/Users/jpec/opt/anaconda3/envs/tf2/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/jpec/Prog/OCRpyMAN/notebooks/text_recognition/ctc_utils.py", line 12, in <module>
    from tqdm import tqdm
ModuleNotFoundError: No module named 'tqdm'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jpec/opt/anacon

In [99]:
img_size = (32, 128)
# data = pd.read_pickle('../../pickle/letter_detection_data.pickle')
# df = data['df']
# preprocessed_imgs = data['preprocessed_imgs']

# print("Length: ", len(df))
# print("Imgs length: ", len(preprocessed_imgs))
# print("Img shape: ", preprocessed_imgs[0].shape)

In [100]:
full_df = pd.read_pickle('../../pickle/df.pickle')
print("Length: ", len(full_df))
ctc_df = full_df[['transcription', 'word_img_path', 'word_id']]
ctc_df.head(3)

Length:  96420


,transcription,word_img_path,word_id
0,A,../data/words/a01/a01-000u/a01-000u-00-00.png,a01-000u-00-00
1,MOVE,../data/words/a01/a01-000u/a01-000u-00-01.png,a01-000u-00-01
2,to,../data/words/a01/a01-000u/a01-000u-00-02.png,a01-000u-00-02


In [127]:
reload(utils)

utils.generate_preprocessed_imgs_from_df(ctc_df)

11252it [1:38:23,  1.60s/it]

In [ ]:
from os.path import exists
ctc_df_path = '../../pickle/ctc_df.pickle'

if exists(ctc_df_path):
    df = pd.read_pickle(ctc_df_path)
else:
    
    pickle.dump(df, open(ctc_df_path, "wb" ))


In [7]:
df.head()

,index,michelson_contrast,gray_level_mot,word_id,gray_level,x,y,w,h,transcription,word_img_path,form_img_path,length
0,944,0.758621,0.889522,a01-020u-06-05,170,1954,1831,389,100,studying,../data/words/a01/a01-020u/a01-020u-06-05.png,../data/formsA-D/a01-020u.png,8
1,914,0.695946,0.774799,a01-020u-02-06,169,1565,1129,128,49,has,../data/words/a01/a01-020u/a01-020u-02-06.png,../data/formsA-D/a01-020u.png,3
2,581,0.683168,0.830613,a01-011u-04-05,166,1622,1472,321,62,discuss,../data/words/a01/a01-011u/a01-011u-04-05.png,../data/formsA-D/a01-011u.png,7
3,357,0.746575,0.817196,a01-007u-00-00,165,402,772,233,60,Since,../data/words/a01/a01-007u/a01-007u-00-00.png,../data/formsA-D/a01-007u.png,5
4,200,0.767606,0.810634,a01-003u-06-07,163,2162,1888,155,69,put,../data/words/a01/a01-003u/a01-003u-06-07.png,../data/formsA-D/a01-003u.png,3


## Model Doc

### CNN Part


The idea here is to cut our image into several features (could be interpreted as smaller areas of the picture).

<img src="imgs/cnn_result.png" />

To do so, we ill use a succession of 
- **Convolution** to extract
- **Batch Normalization** to prevent our model from overfitting (equivalent of Dropout) 

#### Batch normalization

TLDR: Solve **internal covariate shift** and simplify 

**Normalization** is a procedure to change the value of the numeric variable in the dataset to a typical scale, without misshaping contrasts in the range of value.

**Batch normalization** is a technique for training very deep neural networks that normalizes the contributions to a layer for every mini-batch.

In neural networks, the output of the first layer feeds into the second layer, the output of the second layer feeds into the third, and so on. When the parameters of a layer change, so does **the distribution of inputs to subsequent layers**.

> We define Internal Covariate Shift as the change in the distribution of network activations due to the change in network parameters during training.

These shifts in input distributions can be problematic for neural networks, especially deep neural networks that could have a large number of layers.

Batch normalization is a method intended to mitigate internal covariate shift for neural networks.

<img src="imgs/batch_normalization.jpeg" />

This has the impact of settling the learning process and drastically decreasing the number of training epochs required to train deep neural networks.

Sources: 
- https://machinelearning.wtf/terms/internal-covariate-shift/
- https://towardsdatascience.com/batch-normalisation-in-deep-neural-network-ce65dd9e8dbf#:~:text=Batch%20normalization%20solves%20a%20major,you%20can%20often%20remove%20dropout.
- https://towardsdatascience.com/understanding-dataset-shift-f2a5a262a766

#### Conv

<img src="imgs/conv_padding.gif" />

- Padding '**valid**' is the first figure. The filter window stays inside the image. When padding == "VALID", there can be a loss of information. Generally, elements on the right and the bottom of the image tend to be ignored. How many elements are ignored depends on the size of the kernel and the stride.

- Padding '**same**' is the third figure. The output is the same size.


#### Pooling/Flattening

**Pooling** is the process of merging. So it’s basically for the purpose of **reducing the size of the data**.

<img src="imgs/pooling_flattening.png" />

**Flattening** is converting the data into a 1-dimensional array for inputting it to the next layer. We flatten the output of the convolutional layers to create a single long feature vector.

Sources: 

- https://towardsdatascience.com/the-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480

## Model build

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, LeakyReLU, Dropout


hidden_layer_count = 256

def build_sequential_model():
    model = tf.keras.Sequential()
    # Layer 1
    # (None, 32, 128, 64)
    model.add(Conv2D(
            filters=32,
            kernel_size=(5,5),
            padding='SAME',
            input_shape = (img_size[0], img_size[1], 1)
        )
    )
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # Layer 2
    # (None, 16, 64, 64)
    model.add(Conv2D(filters=64, kernel_size=(5,5), padding='SAME'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # Layer 3
    # (None, 8, 32, 128 -> nb conv filters)
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2,1), strides=(2,1)))


    # Layer 4
    # (None, 4, 32, 128)
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2,1), strides=(2,1)))

    # Layer 5
    # (None, 2, 32, 128)
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2,1), strides=(2,1)))
    # (None, 1, 32, 256)
    return model
    
# model = build_sequential_model()
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 128, 32)       832       
                                                                 
 batch_normalization (BatchN  (None, 32, 128, 32)      128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 128, 32)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 64, 64)        51264     
                                                                 
 batch_normalization_1 (Batc  (None, 16, 64, 64)       2

2022-10-16 16:04:38.103287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


At this point, we have cut our entry image into X smaller parts. Those parts can be read as a time serie.

Indeed, there is a strong connection between one part and the one coming after by construction (a word is a succession of letters).

We can therefore make use of a **RNN** now in our model.

NB: a part does not correspond exactly to a letter, it is more the idea behind it. It could be interesting to see the importance of the size of each part (number of cuts).


### RNN part

In [5]:
print(tf.__version__)

2.9.1


In [6]:
from tensorflow.keras.layers import GRU, Bidirectional, Dense, Lambda

model = build_sequential_model()
model.add(Lambda(lambda x: tf.squeeze(x, axis=1)))
# (None, 32, 256)
# Bidirectionnal RNN
model.add(Bidirectional(GRU(hidden_layer_count, return_sequences=True)))
model.add(Dense(100))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 32, 128, 32)       832       
                                                                 
 batch_normalization_5 (Batc  (None, 32, 128, 32)      128       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 128, 32)       0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 64, 64)        51264     
                                                                 
 batch_normalization_6 (Batc  (None, 16, 64, 64)      

```
NotImplementedError: Cannot convert a symbolic Tensor (bidirectional_3/forward_gru_3/strided_slice:0) to a numpy array.
```



TODO LSTM could be a good idea ? 

https://towardsdatascience.com/intuitively-understanding-connectionist-temporal-classification-3797e43a86c
https://distill.pub/2017/ctc/

## Connectionist Temporal Classification

In [50]:
import string

char_list = list(string.ascii_letters)+[' '] 

def extract_allowed_chars_from_string(char_list, str):
    res = ''
    for letter in str:
        if letter in char_list:
            res += letter
    return res

df['clean_trans'] = df.transcription.apply(lambda x: extract_allowed_chars_from_string(char_list, x))
df.head()

,index,michelson_contrast,gray_level_mot,word_id,gray_level,x,y,w,h,transcription,word_img_path,form_img_path,length,clean_trans
0,944,0.758621,0.889522,a01-020u-06-05,170,1954,1831,389,100,studying,../data/words/a01/a01-020u/a01-020u-06-05.png,../data/formsA-D/a01-020u.png,8,studying
1,914,0.695946,0.774799,a01-020u-02-06,169,1565,1129,128,49,has,../data/words/a01/a01-020u/a01-020u-02-06.png,../data/formsA-D/a01-020u.png,3,has
2,581,0.683168,0.830613,a01-011u-04-05,166,1622,1472,321,62,discuss,../data/words/a01/a01-011u/a01-011u-04-05.png,../data/formsA-D/a01-011u.png,7,discuss
3,357,0.746575,0.817196,a01-007u-00-00,165,402,772,233,60,Since,../data/words/a01/a01-007u/a01-007u-00-00.png,../data/formsA-D/a01-007u.png,5,Since
4,200,0.767606,0.810634,a01-003u-06-07,163,2162,1888,155,69,put,../data/words/a01/a01-003u/a01-003u-06-07.png,../data/formsA-D/a01-003u.png,3,put


In [76]:
from sklearn.model_selection import train_test_split

resized_imgs = preprocessed_imgs.reshape(-1, img_size[0], img_size[1])
print(resized_imgs.shape)
X_train, X_test, y_train, y_test = train_test_split(resized_imgs, df.clean_trans.values, test_size=0.1)

(1000, 32, 128)


### Encoder/Decoder

In [68]:
def encode_labels(labels, char_list):
    # Hash Table
    table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            char_list,
            np.arange(len(char_list)),
            value_dtype=tf.int32
        ),
        -1,
        name='char2id'
    )
    return table.lookup(
    tf.compat.v1.string_split(labels, delimiter=''))

def decode_codes(codes, charList):
    table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            np.arange(len(charList)),
            charList,
            key_dtype=tf.int32
        ),
        '',
        name='id2char'
    )
    return table.lookup(codes)

transcrip_test = y_train[:5]
encoded_labels = encode_labels(transcrip_test, char_list)
decoded_codes = decode_codes(encoded_labels, char_list)
print('transcrip_test', transcrip_test)
print('encoded_labels', encoded_labels)
print('-----------------')
print('decoded_codes', decoded_codes)

tf.sparse.to_dense(decoded_codes)

transcrip_test ['meeting' 'Congress' 'Colonial' 'to' 'the']
encoded_labels SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]
 [1 5]
 [1 6]
 [1 7]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [2 4]
 [2 5]
 [2 6]
 [2 7]
 [3 0]
 [3 1]
 [4 0]
 [4 1]
 [4 2]], shape=(28, 2), dtype=int64), values=tf.Tensor(
[12  4  4 19  8 13  6 28 14 13  6 17  4 18 18 28 14 11 14 13  8  0 11 19
 14 19  7  4], shape=(28,), dtype=int32), dense_shape=tf.Tensor([5 8], shape=(2,), dtype=int64))
-----------------
decoded_codes SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]
 [1 5]
 [1 6]
 [1 7]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [2 4]
 [2 5]
 [2 6]
 [2 7]
 [3 0]
 [3 1]
 [4 0]
 [4 1]
 [4 2]], shape=(28, 2), dtype=int64), values=tf.Tensor(
[b'm' b'e' b'e' b't' b'i' b'n' b'g' b'C' b'o' b'n' b'g' b'r' b'e' b's'
 b's' b'C' b'o' b'l' b'o' b'n' b'i' b'a' b'l' b't' b'o' b't' b'h' b'e'], shape=(28,), dtype=string

<tf.Tensor: shape=(5, 8), dtype=string, numpy=
array([[b'm', b'e', b'e', b't', b'i', b'n', b'g', b''],
       [b'C', b'o', b'n', b'g', b'r', b'e', b's', b's'],
       [b'C', b'o', b'l', b'o', b'n', b'i', b'a', b'l'],
       [b't', b'o', b'', b'', b'', b'', b'', b''],
       [b't', b'h', b'e', b'', b'', b'', b'', b'']], dtype=object)>

In [77]:
dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(X_train,-1), y_train))
dataset = dataset.shuffle(1000).batch(64)

In [70]:
def loss(labels, logits):
    return tf.reduce_mean(
            tf.nn.ctc_loss(
                labels = labels,
                logits = logits,
                logit_length = [logits.shape[1]]*logits.shape[0],
                label_length = None,
                logits_time_major = False,
                blank_index=-1
            )
        )

        
def train_op(model, optimizer, inputs, targets):
    with tf.GradientTape() as tape:
        # Prédiction de notre modèle
        y_pred = model(inputs, training=True)
        # Calcule de l'erreur de notre modèle
        loss_value = tf.reduce_mean(loss(targets, y_pred))
       
    # Calculer le gradient de la fonction de perte
    grads = tape.gradient(loss_value, model.trainable_variables)
    # Descente de gradient
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # Retourner la valeur de la fonction de perte
    return loss_value.numpy()



In [78]:
import sys
import time

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
epochs = 10

# Entraînenement du modèle
for i in range(epochs): 
    # Pour chaque epoch
    print('---- Epoch', i, '----')
    j = 0
    for X_b, y_b in dataset:

        try :
            y_ba = encode_labels(y_b, char_list)
            print("Batch {}".format(j), train_op(model, optimizer, X_b, y_ba))
        except :
            print("Unexpected error:", sys.exc_info()[0])
            time.sleep(0.5)
        j += 1

---- Epoch 0 ----
Batch 0 14.908546
Batch 1 13.797148
Batch 2 12.310874
Batch 3 13.523983
Batch 4 12.716388
Batch 5 13.529551
Batch 6 13.058186
Batch 7 11.762482
Batch 8 12.022669
Batch 9 13.708242
Batch 10 12.919116
Batch 11 12.013939
Batch 12 12.237228
Batch 13 11.8152685
Batch 14 10.614099
---- Epoch 1 ----
Batch 0 11.830199
Batch 1 12.390379
Batch 2 12.108862
Batch 3 12.791401
Batch 4 11.464293
Batch 5 15.061083
Batch 6 11.8678465
Batch 7 13.125231
Batch 8 13.699672
Batch 9 12.20892
Batch 10 12.323076
Batch 11 11.666872
Batch 12 11.606966
Batch 13 12.071641
Batch 14 10.583216
---- Epoch 2 ----
Batch 0 13.369407
Batch 1 10.80093
Batch 2 13.413527
Batch 3 11.385896
Batch 4 10.460476
Batch 5 11.662494
Batch 6 14.186923
Batch 7 13.483528
Batch 8 11.593671
Batch 9 12.272942
Batch 10 10.875294
Unexpected error: <class 'KeyboardInterrupt'>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you w

In [75]:
def greedy_decoder(logits):
    # ctc beam search decoder
    predicted_codes, _ = tf.nn.ctc_greedy_decoder(
        # shape of tensor [max_time x batch_size x num_classes] 
        tf.transpose(logits, (1, 0, 2)),
        [logits.shape[1]]*logits.shape[0]
    )
    
    # convert to int32
    codes = tf.cast(predicted_codes[0], tf.int32)
    
    # Decode the index of caracter
    text = decode_codes(codes, char_list)
    
    # Convert a SparseTensor to string
    text = tf.sparse.to_dense(text).numpy().astype(str)
    
    return list(map(lambda x: ''.join(x), text))

predicted_transcriptions = greedy_decoder(model(np.expand_dims(X_test[:10], -1)))
list(zip(predicted_transcriptions, y_test[:10]))

[('', 'conference'),
 ('', 'the'),
 ('', 'Independence'),
 ('', 'and'),
 ('', 'turn'),
 ('', 'chief'),
 ('', 'together'),
 ('', 'Chiefs'),
 ('', 'Most'),
 ('', 'up')]